In [1]:
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
import numpy as np
import pickle
import sys
np.seterr(divide="ignore")
sys.path.append("..")
from recnn.recnn import grnn_predict_simple
from recnn.recnn import grnn_predict_gated
from recnn.preprocessing import rewrite_content
from recnn.preprocessing import multithreadmap
from recnn.preprocessing import permute_by_pt
from recnn.preprocessing import extract

In [2]:
def extractcontent(jet):
    return(jet["content"])

def tftransform(jet,tf):
    """applies a robustscaler transform"""
    jet["content"] = tf.transform(jet["content"])
    return(jet)

def load_tf(X, y):
    """loads training data and make a robustscaler transform"""
    # Make training data
    X = multithreadmap(rewrite_content,X)
    X = multithreadmap(extract,X)
    Xcontent=multithreadmap(extractcontent,X)
    tf = RobustScaler().fit(np.vstack(Xcontent))
    return(tf)

def load_test(tf, X, y):
    """loads testing data"""
    # Make test data
    shuf = np.random.permutation(len(X))
    X=X[shuf]
    y=y[shuf]
    print("Preprocessing...")
    X = multithreadmap(rewrite_content,X)
    X = multithreadmap(permute_by_pt,X)
    X = multithreadmap(extract,X)
    X=multithreadmap(tftransform,X,tf=tf)
    return(X, y)

def roc_curve_perso(y, y_pred,density=1000):
    """return the roc curve"""
    back = np.argwhere(y==0)
    back = back.reshape((len(back),))
    sign = np.argwhere(y==1)
    sign=sign.reshape((len(sign),))
    #prediction
    y_pred_sign=y_pred[sign]
    y_pred_back=y_pred[back]
    t=np.linspace(0.,1.,density)
    tpr=np.zeros(density,dtype=float)
    fpr=np.zeros(density,dtype=float)
    for i in range(density):
        tpr[i]=np.sum(y_pred_sign<=t[i])
        fpr[i]=np.sum(y_pred_back<=t[i])
    tpr=1-tpr/len(y_pred_sign)
    fpr=1-fpr/len(y_pred_back)
    return(fpr,tpr,t)

In [3]:
def predict(X, filename, func=grnn_predict_simple):
    """make prediction function"""
    fd = open(filename, "rb")
    params = pickle.load(fd)
    fd.close()
    y_pred = func(params, X)
    return(y_pred)

def evaluate_models(X, y, filename, func=grnn_predict_simple):
    """evaluates a model"""
    print("Loading " + filename),
    y_pred = predict(X, filename, func=func)
    fpr, tpr, _ = roc_curve_perso(y, y_pred,density=10000)
    roc = np.trapz(-tpr,fpr)
    print("ROC AUC = %.4f" % roc)
    return(roc, fpr, tpr)

def build_rocs(tf, X1, y1, model):
    X, y = load_test(tf, X1, y1) 
    roc, fpr, tpr = evaluate_models(X, y, model, func=grnn_predict_gated)
    return(roc, fpr, tpr)

In [4]:
basepath = '/data/conda/recnn/data'
name="anti-kt"
trainfile,testfile = basepath+"/npyfiles/subjet_oriented_"+name+"_train.npy",basepath+"/npyfiles/subjet_oriented_"+name+"_test.npy"
modelpath = basepath+"/models/subjet_oriented_"+name+"_model.pickle"

In [5]:
### Load training data ###
X, y = np.load(trainfile)
X=np.array(X).astype(dict)
y = np.array(y).astype(int)

### to rescale test data ###
tf = load_tf(X,y)

### Load test data ###
X1, y1 = np.load(testfile)
X1 = np.array(X1).astype(dict)
y1 = np.array(y1).astype(int)

In [6]:
### Build the roc ###
r, f, t = build_rocs(tf, X1, y1, basepath+modelpath)
print(r)

Preprocessing...
Loading /data/conda/recnn/data/data/conda/recnn/data/models/subjet_oriented_anti-kt_model.pickle

IOError: [Errno 2] No such file or directory: '/data/conda/recnn/data/data/conda/recnn/data/models/subjet_oriented_anti-kt_model.pickle'

In [7]:
plt.plot(f,t,label=name)
tpr,fpr = np.load('/data/conda/recnn/data/roccurves/standardID_ROC.npy')
plt.plot(fpr[1:],tpr[1:],label='std')
#plt.xscale("log")
plt.legend()
plt.grid()
#plt.yscale("log")
#plt.xlim([0.0001,0.1])
#plt.ylim([0.2,0.8])
#plt.savefig("first_working_nn_roc.png",dpi=600)
print(r)

NameError: name 'f' is not defined

In [ ]:
basepath = '/data/conda/recnn/data'
filelist=["anti-kt","kt","random","seqpt","seqpt_reversed","cambridge"]
#filelist=["seqpt"]
roc_list = []

for name in filelist:
    trainfile,testfile = basepath+"/npyfiles/subjet_oriented_"+name+"_train.npy",basepath+"/npyfiles/subjet_oriented_"+name+"_test.npy"
    modelpath = basepath+"/models/subjet_oriented_"+name+"_model.pickle"
    ### Load training data ###
    X, y = np.load(trainfile)
    X=np.array(X).astype(dict)
    y = np.array(y).astype(int)

    ### to rescale test data ###
    tf = load_tf(X,y)

    ### Load test data ###
    X1, y1 = np.load(testfile)
    X1 = np.array(X1).astype(dict)
    y1 = np.array(y1).astype(int)
    ### Build the roc ###
    r, f, t = build_rocs(tf, X1, y1, modelpath)
    print(r)
    roc_list.append((f,t,name))
    print(r)


for name in filelist:
    trainfile,testfile = basepath+"/npyfiles/particle_oriented_"+name+"_train.npy",basepath+"/npyfiles/particle_oriented_"+name+"_test.npy"
    modelpath = basepath+"/models/particle_oriented_"+name+"_model.pickle"
    ### Load training data ###
    X, y = np.load(trainfile)
    X=np.array(X).astype(dict)
    y = np.array(y).astype(int)

    ### to rescale test data ###
    tf = load_tf(X,y)

    ### Load test data ###
    X1, y1 = np.load(testfile)
    X1 = np.array(X1).astype(dict)
    y1 = np.array(y1).astype(int)
    ### Build the roc ###
    r, f, t = build_rocs(tf, X1, y1, modelpath)
    print(r)
    roc_list.append((f,t,name))
    print(r)


 Preprocessing...
Loading /data/conda/recnn/data/models/subjet_oriented_anti-kt_model.pickle ROC AUC = 0.9571
0.9571228814233819
0.9571228814233819
Preprocessing...
Loading /data/conda/recnn/data/models/subjet_oriented_kt_model.pickle ROC AUC = 0.9592
0.9592418928744646
0.9592418928744646
Preprocessing...
Loading /data/conda/recnn/data/models/subjet_oriented_random_model.pickle ROC AUC = 0.9588
0.9587682889296372
0.9587682889296372
Preprocessing...
Loading /data/conda/recnn/data/models/subjet_oriented_seqpt_model.pickle ROC AUC = 0.9613
0.9613179019548661
0.9613179019548661
Preprocessing...


In [ ]:
tpr,fpr = np.load('/data/conda/recnn/data/roccurves/standardID_ROC.npy')
roc_list.append((fpr[1:],tpr[1:],"std"))

In [ ]:
for (i,j,k) in roc_list :
    plt.plot(i,j,label=k)
    #plt.xscale("log")
    plt.legend()
    plt.grid()
    #plt.yscale("log")
    #plt.xlim([0.001,0.1])
    #plt.ylim([0.2,0.8])
    plt.savefig("first_comparisions_dezoom.png",dpi=600)
plt.show()